# 환경 변수 로드

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 6-1

In [13]:
from langchain import hub
from langchain_community.tools import TavilySearchResults
from langchain_anthropic import ChatAnthropic
from langchain.agents import create_tool_calling_agent

# hub에서 'hwchase17/openai-functions-agent' 프롬프트를 불러옴
prompt = hub.pull("hwchase17/openai-functions-agent")

# 프롬프트 메시지를 출력
print("prompt:")
print(prompt.messages)
print("-" * 100)

# Tavily 검색 결과 도구를 생성하고 최대 결과를 2개로 제한
web_search = TavilySearchResults(max_results=2)
tools = [web_search]

# Claude-3 모델을 사용하여 ChatAnthropic 모델 생성
model = ChatAnthropic(model_name="claude-3-haiku-20240307")

# 도구 호출 에이전트를 생성하여, 프롬프트와 도구들을 에이전트에 결합
agent = create_tool_calling_agent(model, tools, prompt)

# 에이전트를 출력
print("agent:")
print(agent)
print("-" * 100)

/Users/steve2/Library/Caches/pypoetry/virtualenvs/langchain-book-AlsLlHcI-py3.11/lib/python3.11/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


prompt:
[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]
----------------------------------------------------------------------------------------------------
agent:
first=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
}) middle=[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag

# 6-2

In [14]:
from langchain.agents import AgentExecutor

# 에이전트와 도구를 결합하여 AgentExecutor 객체 생성
agent_executor = AgentExecutor(agent=agent, tools=tools) 

# AgentExecutor 객체 출력
print(agent_executor)

verbose=False agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')],

# 6-3

In [15]:
# 첫 번째 예시: 도구를 호출할 필요가 없을 때 에이전트의 응답
agent_executor.invoke({"input": "안녕하세요!"})

{'input': '안녕하세요!',
 'output': [{'text': '네, 안녕하세요! 무엇을 도와드릴까요?', 'type': 'text', 'index': 0}]}

# 6-4

In [16]:
# 웹 검색 도구를 호출해야 하는 예시를 실행
agent_executor.invoke({"input": "2024년 하반기 애플 신제품은 무엇인가요?"})

{'input': '2024년 하반기 애플 신제품은 무엇인가요?',
 'output': [{'text': '\n\n2024년 하반기에 애플이 출시할 것으로 예상되는 신제품은 다음과 같습니다:\n\n- 애플 비전 프로(Apple Vision Pro) - 새로운 AR/VR 헤드셋 제품\n- 아이폰 16 - 애플의 신형 스마트폰\n- 아이패드 프로 OLED - OLED 디스플레이 적용된 프리미엄 태블릿 라인업\n- 에어팟 4 - 애플의 무선 이어폰 새로운 세대\n- 애플워치 10 - 애플워치의 10세대 모델로 주요 건강 기능 업그레이드 예정\n\n주요 제품들의 성능, 디자인, 가격 등 구체적인 사양 정보는 정식 출시 발표에 맞춰 알려질 것으로 보입니다. 애플은 신제품 라인업을 통해 2024년 하반기에 다양한 고객 니즈를 충족시키고자 노력할 것으로 예상됩니다.',
   'type': 'text',
   'index': 0}]}

# 6-5

In [55]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# session_id를 기반으로 대화 기록을 가져오거나 새로 생성하는 함수
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# store 출력
print(store)

{}


# 6-6

In [57]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

# 메모리를 갖춘 에이전트 생성
agent_with_chat_history = RunnableWithMessageHistory(
     # 기존 에이전트 실행도구
    agent_executor | (lambda x: AIMessage(content=x["output"][0]["text"], role="assistant")),               
    get_session_history,           # 세션별 대화 기록을 관리하는 함수
    input_messages_key="input",    # 입력 대화를 저장할 키
    history_messages_key="chat_history"  # 대화 기록을 불러올 키
)

# 세션 ID를 지정
config = {"configurable": {"session_id": "test_001"}}  

# 첫 번째 대화: 새로운 세션에서 질문 
response = agent_with_chat_history.invoke(
    {"input": "제 이름은 판다스입니다. 테슬라 회장은 누구인가요?"}, 
    config=config
)

# 에이전트의 응답 출력
print("response: ")
print(response.content)
print("-"*100)

# store 출력
print("store: ")
print(store)
print("-"*100)

response: 


테슬라의 CEO이자 창업자는 일론 머스크(Elon Musk)입니다. 일론 머스크는 2008년부터 테슬라의 CEO를 맡고 있으며, 스페이스X, 솔라시티, 트위터, 더 보링 컴퍼니, 뉴런링크 등 다양한 회사를 이끌고 있는 기업가입니다. 그는 지속 가능한 에너지 기업인 테슬라를 이끌면서 전기 자동차, 배터리, 태양 에너지 사업을 성공적으로 펼치고 있습니다.
----------------------------------------------------------------------------------------------------
store: 
{'test_001': InMemoryChatMessageHistory(messages=[HumanMessage(content='제 이름은 판다스입니다. 테슬라 회장은 누구인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n테슬라의 CEO이자 창업자는 일론 머스크(Elon Musk)입니다. 일론 머스크는 2008년부터 테슬라의 CEO를 맡고 있으며, 스페이스X, 솔라시티, 트위터, 더 보링 컴퍼니, 뉴런링크 등 다양한 회사를 이끌고 있는 기업가입니다. 그는 지속 가능한 에너지 기업인 테슬라를 이끌면서 전기 자동차, 배터리, 태양 에너지 사업을 성공적으로 펼치고 있습니다.', additional_kwargs={}, response_metadata={}, role='assistant')])}
----------------------------------------------------------------------------------------------------


# 6-7

In [58]:
# 이전 대화를 기억하는지 확인
response = agent_with_chat_history.invoke(  
    {"input": "제 이름은 무엇인가요?"},
    config=config
)

# 에이전트의 응답 출력
print("response: ")
print(response.content)
print("-"*100)

# store 출력
print("store: ")
print(store)
print("-"*100)

response: 
죄송합니다. 제가 처음에 잘못 알아들었던 것 같습니다. 여러분의 이름이 판다스라고 말씀하셨네요. 감사합니다!
----------------------------------------------------------------------------------------------------
store: 
{'test_001': InMemoryChatMessageHistory(messages=[HumanMessage(content='제 이름은 판다스입니다. 테슬라 회장은 누구인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\n테슬라의 CEO이자 창업자는 일론 머스크(Elon Musk)입니다. 일론 머스크는 2008년부터 테슬라의 CEO를 맡고 있으며, 스페이스X, 솔라시티, 트위터, 더 보링 컴퍼니, 뉴런링크 등 다양한 회사를 이끌고 있는 기업가입니다. 그는 지속 가능한 에너지 기업인 테슬라를 이끌면서 전기 자동차, 배터리, 태양 에너지 사업을 성공적으로 펼치고 있습니다.', additional_kwargs={}, response_metadata={}, role='assistant'), HumanMessage(content='제 이름은 무엇인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='죄송합니다. 제가 처음에 잘못 알아들었던 것 같습니다. 여러분의 이름이 판다스라고 말씀하셨네요. 감사합니다!', additional_kwargs={}, response_metadata={}, role='assistant')])}
----------------------------------------------------------------------------------------------------


# 6-8

In [66]:
from langgraph.prebuilt import create_react_agent

# LLM에 도구 바인딩하여 ReAct 에이전트 생성 
agent = create_react_agent(model, tools=tools)

# 입력 값 정의 
inputs = {"messages": [("user", "2024년 테슬라 회장은 누구인가요? 한국어로 답변하세요.")]}

# 에이전트의 응답을 스트리밍 방식으로 출력  
for s in agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

2024년 테슬라 회장은 누구인가요? 한국어로 답변하세요.
================================== Ai Message ==================================

[{'id': 'toolu_013bo1nBEwoAgQoeZhUMcMCs', 'input': {'query': '2024년 테슬라 회장'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_013bo1nBEwoAgQoeZhUMcMCs)
 Call ID: toolu_013bo1nBEwoAgQoeZhUMcMCs
  Args:
    query: 2024년 테슬라 회장
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.chosun.com/economy/tech_it/2024/02/02/IT65WYOF6FF5NCKXANQPZU57Q4/", "content": "2024년 9월 26일(목) ... 지난 2018년 테슬라 이사회는 머스크에 대해 560억달러 규모의 보상 패키지 지급안을 승인했다. 이후 테슬라 주식 9주를 가진 소액 주주 리처드 토네타는 \"이사회가 승인한 보상 패키지는 역사상 유례없는 수준의 고액으로 과도하게 많다\"며 ..."}, {"url": "https://biz.chosun.com/international/international_general/2024/06/14/CGTTU7MPJBCDJA5OMO3NSKLBME/", "content": "테슬라 주주총회서 67兆 규모 머스